<a href="https://colab.research.google.com/github/sruthi2498/Arduino-based-smart-glove/blob/master/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import re
import pandas as pd
import string
import json
import requests
import numpy as np

In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras import optimizers, Model
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, SpatialDropout1D, LSTM, Bidirectional, TimeDistributed, Input
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils.vis_utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs

In [2]:
url = "https://raw.githubusercontent.com/paresh-nayyar/CSCI-544_NLP/master/data/Towards_Emotion_Recognition_text%26emotions.csv?token=GHSAT0AAAAAABQE6M6KU545Y63IPZNPX34OYSXJ4EQ"
df = pd.read_csv(url)
df.head()

,tweet_id,tweet,anger,disgust,fear,happiness,sadness,surprise
0,46149,don ko11mulko ki police sms kar rahi hai par d...,0,0,0,1,0,0
1,40317002,khushi ki talaash me sabhi haiyn kya maiyn aur...,0,0,0,0,1,0
2,40319032,are dostoin aao aur apne thoughts share karo l...,0,0,0,0,1,0
3,308624342,na tum aa sako gay na phir baat hogi yeh kaisi...,0,0,0,0,1,0
4,596311632,meethe gud me mil gaye til udi patang aur khil...,0,0,0,1,0,0


In [3]:
from google.colab import drive
drive.mount('/content/drive')
!ls

Mounted at /content/drive
drive  sample_data


In [4]:
%cd drive/My Drive/
# !ls

/content/drive/My Drive
'Applied NLP App Idea : Vernacular Programming.gdoc'
 code_mix_sentences_emotions_tagged.xlsx
'Colab Notebooks'
 fasttext_model
'Honors Statement.gdoc'
 I-20_Venkatavaradan_Sruthi_4_N0031930432_1638314915746.pdf
 MusicAlbumSimilarity.gdoc
 NLI_training
 PersonalStatement.pdf
'question papers.zip'
'RL question.gdoc'
'SAIL_CodeMixed_2017_results (1).csv'
'SruthiVenkatavaradan_DS (1).pdf'
'SruthiVenkatavaradan_DS (2).pdf'
'SruthiVenkatavaradan_DS (3).pdf'
'SruthiVenkatavaradan_DS (4).pdf'
 SruthiVenkatavaradan_DS.pdf
'Sruthi-Venkatavaradan Honors Program LOR.gdoc'
 SruthiVenkatavaradan_SE.pdf
'Untitled document.gdoc'
'Untitled spreadsheet (1).gsheet'
'Untitled spreadsheet.gsheet'
 USC


In [5]:
from gensim.models import FastText
fasttext_url = "./fasttext_model/fasttext_model.model"
embeddings_index = FastText.load(fasttext_url)

In [6]:
train_df = df
test_df= df

In [9]:
label_names = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']
y_train = train_df[label_names].values
y_test = test_df[label_names].values

train_df['doc_len'] = train_df['tweet'].astype(str).apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
#max_seq_len = train_df['doc_len'].max()
print(max_seq_len)


28


In [10]:
processed_docs_train = train_df['tweet'].astype(str).tolist()
processed_docs_test = test_df['tweet'].astype(str).tolist() 
num_classes = len(label_names)


In [14]:
MAX_NB_WORDS = 200000
tokenizer = RegexpTokenizer(r'\w+')

In [15]:
print("tokenizing input data...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
total_docs=[]
total_docs.extend(processed_docs_train)
total_docs.extend(processed_docs_test)
#tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)  #leaky
tokenizer.fit_on_texts(total_docs)
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
#print(word_seq_train)
word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_index = tokenizer.word_index
print(len(word_index))
print("dictionary size: ", len(word_index))

#pad sequences
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
print(word_seq_train[2])
word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)

tokenizing input data...
235929
dictionary size:  235929
[    0     0   262 72837   908    12    77  2536   746   137   584   512
   243    12    17     6   963    14 72838   649   205     7 72839  1315
  2570   166   842  1229]


In [16]:
#training params
batch_size = 64
num_epochs = 8 

#model parameters
num_filters = 128 
num_kernels=200
stride=1
embed_dim = 300
weight_decay = 1e-4

#parallel layers 
kernel_size=7
kernel_size_p1=3
kernel_size_p2=6
kernel_size_p3=9
kernel_size_p4=12

In [17]:
#embedding matrix
print('preparing embedding matrix...')
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    #print(word)

    if embeddings_index.wv.__contains__(word): 

        embedding_vector = embeddings_index.wv[word]
    
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
#print(embedding_matrix)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

preparing embedding matrix...
number of null word embeddings: 1


In [21]:
print(len(embedding_matrix[34]))
print(words_not_found)


300
[]


In [23]:
print("training CNN ...")
model = Sequential()
model.add(Embedding(nb_words, embed_dim,
          weights=[embedding_matrix], input_length=max_seq_len, trainable=False))
model.add(Conv1D(num_kernels, kernel_size, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_kernels, kernel_size, activation='relu', padding='same'))
#model.add(GlobalMaxPooling1D())
model.add(MaxPooling1D(2))
#model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
#model.add(Dense(num_classes, activation='sigmoid'))  #multi-label (k-hot encoding)
model.add(Dense(num_classes, activation='sigmoid')) 

# adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

training CNN ...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 28, 300)           60000000  
                                                                 
 conv1d_2 (Conv1D)           (None, 28, 200)           420200    
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 14, 200)          0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 14, 200)           280200    
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 7, 200)           0         
 1D)                                                             
                                                                 
 global_max_pooling1d_1 (Glo  (None, 

In [24]:
hist = model.fit(word_seq_train, y_train, batch_size=batch_size, epochs=num_epochs, validation_split=0.1, shuffle=True)


Epoch 1/8
1895/1895 [==============================] - 353s 186ms/step - loss: 0.2234 - accuracy: 0.9265 - val_loss: 0.2323 - val_accuracy: 0.9221
Epoch 2/8
1895/1895 [==============================] - 315s 166ms/step - loss: 0.1350 - accuracy: 0.9575 - val_loss: 0.1882 - val_accuracy: 0.9366
Epoch 3/8
1895/1895 [==============================] - 313s 165ms/step - loss: 0.1192 - accuracy: 0.9625 - val_loss: 0.1696 - val_accuracy: 0.9437
Epoch 4/8
1895/1895 [==============================] - 299s 158ms/step - loss: 0.1131 - accuracy: 0.9634 - val_loss: 0.1877 - val_accuracy: 0.9442
Epoch 5/8
1895/1895 [==============================] - 282s 149ms/step - loss: 0.1053 - accuracy: 0.9662 - val_loss: 0.1673 - val_accuracy: 0.9454
Epoch 6/8
1895/1895 [==============================] - 294s 155ms/step - loss: 0.1014 - accuracy: 0.9675 - val_loss: 0.1390 - val_accuracy: 0.9526
Epoch 7/8
1895/1895 [==============================] - 310s 164ms/step - loss: 0.0962 - accuracy: 0.9689 - val_loss: 0